## SpaceX Falcon 9 First Stage Landing Prediction

### 1. Collecting Data

In [1]:
# importing required lobraries

import requests
import pandas as pd 
import numpy as np 
import datetime

In [4]:
# setting that to print  all columns and all data of a dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [23]:
data = requests.get("https://api.spacexdata.com/v4/rockets/").json()
data

[{'height': {'meters': 22.25, 'feet': 73},
  'diameter': {'meters': 1.68, 'feet': 5.5},
  'mass': {'kg': 30146, 'lb': 66460},
  'first_stage': {'thrust_sea_level': {'kN': 420, 'lbf': 94000},
   'thrust_vacuum': {'kN': 480, 'lbf': 110000},
   'reusable': False,
   'engines': 1,
   'fuel_amount_tons': 44.3,
   'burn_time_sec': 169},
  'second_stage': {'thrust': {'kN': 31, 'lbf': 7000},
   'payloads': {'composite_fairing': {'height': {'meters': 3.5, 'feet': 11.5},
     'diameter': {'meters': 1.5, 'feet': 4.9}},
    'option_1': 'composite fairing'},
   'reusable': False,
   'engines': 1,
   'fuel_amount_tons': 3.38,
   'burn_time_sec': 378},
  'engines': {'isp': {'sea_level': 267, 'vacuum': 304},
   'thrust_sea_level': {'kN': 420, 'lbf': 94000},
   'thrust_vacuum': {'kN': 480, 'lbf': 110000},
   'number': 1,
   'type': 'merlin',
   'version': '1C',
   'layout': 'single',
   'engine_loss_max': 0,
   'propellant_1': 'liquid oxygen',
   'propellant_2': 'RP-1 kerosene',
   'thrust_to_weight': 

#### From the ' rocket ' column we will get booster name

In [10]:
# takes dataset and uses the rocket column to call the API and append the data t0 the list
def getBoosterVersion(data):
    for i in data['rocket']:
        if i:
            response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(i)).json()
            BoosterVersion.append([response['name']])

#### From the ' launchpad ' we get the name of the launch site being used, the longitude and the latitude

In [89]:
# takes the dataset and uses the launchpad column to call API and append the data to the list
def getLaunchSite(data):
    for i in data['launchpad']:
        if i:
            response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(i)).json()
            LaunchSite.append(response['name'])
            Longitude.append(response['longitude'])
            Latitude.append(response['latitude'])          

#### From the ' payload ' we will learn the mass of the payload and the orbit that it is going to

In [14]:
# takes the dataset and uses the payload column to call the API and append the data to the list
def getPayloadData(data):
    for i in data['payloads']:
        if i:
            response = requests.get("https://api.spacexdata.com/v4/payloads/"+i).json()
            PayloadMass.append(response['mass_kg'])
            Orbit.append(response['orbit'])

#### From ' cores ' we will learn the outcome of the landing, type of landing, no.of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad is used or not, the bolick of tje core which is a no. wused to seprate version of cores, the no.of times this specific core has been reused and the serial of the core

In [15]:
# takes the dataset and uses the cores column to call the APi and append the data to the list
def getCoreData(data):
    for core in data['cores']:
        if core['core'] != None:
            response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
            Block.append(response['block'])
            ReusedCount.append(responce['reuse_count'])
            Serial.append(response['serial'])
        else:
            Block.append(None)
            Reused.append(None)
            Serial.append(None)
        Outcome.append(str(core['landing_success']) + ' ' + str(core['landing_type']))
        Flights.append(core[flight])
        GridFins.append(core['gridfins'])
        Reused.append(core['reused'])
        Legs.append(core['legs'])
        LandingPad.append(core['landpad'])

### Now lets start request data from SpaceX API 

In [40]:
# requesting rocket launch data 
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [45]:
response = requests.get(spacex_url).json()

In [49]:
# response.content

to make the requested JSON results more consistent we will use static response object

In [27]:
static_json_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'

In [42]:
# checking whether the requests was succesfull or not
response.status_code

200

In [57]:
data = pd.json_normalize(response)

In [58]:
data.head()

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,fairings.reused,fairings.recovery_attempt,fairings.recovered,fairings.ships,links.patch.small,links.patch.large,links.reddit.campaign,links.reddit.launch,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False,[],https://images2.imgbox.com/94/f2/NN6Ph45r_o.png,https://images2.imgbox.com/5b/02/QcxHUb5V_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'harmonic oscillation leading to premature engine shutdown'}]","Successful first stage burn and transition to second stage, maximum altitude 289 km, Premature engine shutdown at T+7 min 30 s, Failed to reach orbit, Failed to recover first stage",[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cdaffd86e000604b32b,False,False,False,[],https://images2.imgbox.com/f9/4a/ZboXReNb_o.png,https://images2.imgbox.com/80/a2/bkWotCIS_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-rocket-fails-reach-orbit.html,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'residual stage-1 thrust led to collision between stage 1 and stage 2'}]",Residual stage 1 thrust led to collision between stage 1 and stage 2,[],[],[],"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006eeb1e4]",5e9e4502f5090995de566f86,3,Trailblazer,2008-08-03T03:34:00.000Z,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cdbffd86e000604b32c,False,False,False,[],https://images2.imgbox.com/6c/cb/na1tzhHs_o.png,https://images2.imgbox.com/4a/80/k1oAkY0k_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1-flight-3-mission-summary,https://en.wikipedia.org/wiki/Trailblazer_(satellite),NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],"Ratsat was carried to orbit on the first successful orbital launch of any privately funded and developed, liquid-propelled carrier rocket, the SpaceX Falcon 1",[],[],[],[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,4,RatSat,2008-09-28T23:15:00.000Z,1222643700,2008-09-28T11:15:00+12:00,hour,False,"[{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': 

we can notice from the above data that are IDs like the rocket column has no information about the rocket it have just an identification number

Now we will retrive the columns ' rocket ', ' pyloads ' . ' launchpad ' and ' cores '

In [62]:
# take a asubset of our dataframe by keeping onthe the features whihc are neccessary
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

In [61]:
data

,rocket,payloads,launchpad,cores,flight_number,date_utc
0,5e9d0d95eda69955f709d1eb,[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,"[{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",1,2006-03-24T22:30:00.000Z
1,5e9d0d95eda69955f709d1eb,[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,"[{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",2,2007-03-21T01:10:00.000Z
2,5e9d0d95eda69955f709d1eb,"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006eeb1e4]",5e9e4502f5090995de566f86,"[{'core': '5e9e289ef3591814873b2625', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",3,2008-08-03T03:34:00.000Z
3,5e9d0d95eda69955f709d1eb,[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,"[{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",4,2008-09-28T23:15:00.000Z
4,5e9d0d95eda69955f709d1eb,[5eb0e4b7b6c3bb0006eeb1e6],5e9e4502f5090995de566f86,"[{'core': '5e9e289ef359184f103b2627', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",5,2009-07-13T03:35:00.000Z
...,...,...,...,...,...,...
182,5e9d0d95eda69973a809d1ec,"[631614e9ffc78f3b85670717, 631617fbffc78f3b8567071d]",5e9e4501f509094ba4566f84,"[{'core': '5e9e28a6f359183c413b265d', 'flight': 7, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecbb9e534e7cc'}]",183,2022-09-05T02:09:00.000Z
183,5e9d0d95eda69973a809d1ec,"[63161610ffc78f3b85670718, 63161872ffc78f3b8567071e]",5e9e4502f509094188566f88,"[{'core': '5e9e28a7f3591817f23b2663', 'flight': 14, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecb075134e7cd'}]",184,2022-09-11T01:10:00.000Z
184,5e9d0d95eda69973a809d1ec,[63161699ffc78f3b85670719],5e9e4501f509094ba4566f84,"[{'core': '60b800111f83cc1e59f16438', 'flight': 6, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecbb9e534e7cc'}]",185,2022-09-17T01:05:00.000Z
185,5e9d0d95eda69973a809d1ec,[631616a7ffc78f3b8567071a],5e9e4501f509094ba4566f84,"[{'core': '627843d657b51b752c5c5a53', 'flight': 4, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecbb9e534e7cc'}]",186,2022-09-24T23:30:00.000Z


In [70]:
# remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

In [71]:
data

,rocket,payloads,launchpad,cores,flight_number,date_utc
0,5e9d0d95eda69955f709d1eb,[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,"[{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",1,2006-03-24T22:30:00.000Z
1,5e9d0d95eda69955f709d1eb,[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,"[{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",2,2007-03-21T01:10:00.000Z
3,5e9d0d95eda69955f709d1eb,[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,"[{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",4,2008-09-28T23:15:00.000Z
4,5e9d0d95eda69955f709d1eb,[5eb0e4b7b6c3bb0006eeb1e6],5e9e4502f5090995de566f86,"[{'core': '5e9e289ef359184f103b2627', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",5,2009-07-13T03:35:00.000Z
5,5e9d0d95eda69973a809d1ec,[5eb0e4b7b6c3bb0006eeb1e7],5e9e4501f509094ba4566f84,"[{'core': '5e9e289ef359185f2b3b2628', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",6,2010-06-04T18:45:00.000Z
...,...,...,...,...,...,...
180,5e9d0d95eda69973a809d1ec,[631614d7ffc78f3b85670716],5e9e4502f509094188566f88,"[{'core': '61c1ef45a4a2462678cbf45d', 'flight': 2, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecb075134e7cd'}]",181,2022-08-28T02:22:00.000Z
181,5e9d0d95eda69973a809d1ec,[630f63bf18702d4844fb5391],5e9e4502f509092b78566f87,"[{'core': '5f57c54a0622a633027900a1', 'flight': 7, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3032383ecb6bb234e7ca'}]",182,2022-08-31T05:40:00.000Z
184,5e9d0d95eda69973a809d1ec,[63161699ffc78f3b85670719],5e9e4501f509094ba4566f84,"[{'core': '60b800111f83cc1e59f16438', 'flight': 6, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecbb9e534e7cc'}]",185,2022-09-17T01:05:00.000Z
185,5e9d0d95eda69973a809d1ec,[631616a7ffc78f3b8567071a],5e9e4501f509094ba4566f84,"[{'core': '627843d657b51b752c5c5a53', 'flight': 4, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecbb9e534e7cc'}]",186,2022-09-24T23:30:00.000Z


In [72]:
# now extract the singel value in the list and replace the feature
data['cores'] = data['cores'].map(lambda x : x[0])
data['payloads'] = data['payloads'].map(lambda x : x[0])

In [73]:
data

,rocket,payloads,launchpad,cores,flight_number,date_utc
0,5e9d0d95eda69955f709d1eb,5eb0e4b5b6c3bb0006eeb1e1,5e9e4502f5090995de566f86,"{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",1,2006-03-24T22:30:00.000Z
1,5e9d0d95eda69955f709d1eb,5eb0e4b6b6c3bb0006eeb1e2,5e9e4502f5090995de566f86,"{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",2,2007-03-21T01:10:00.000Z
3,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e5,5e9e4502f5090995de566f86,"{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",4,2008-09-28T23:15:00.000Z
4,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e6,5e9e4502f5090995de566f86,"{'core': '5e9e289ef359184f103b2627', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",5,2009-07-13T03:35:00.000Z
5,5e9d0d95eda69973a809d1ec,5eb0e4b7b6c3bb0006eeb1e7,5e9e4501f509094ba4566f84,"{'core': '5e9e289ef359185f2b3b2628', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",6,2010-06-04T18:45:00.000Z
...,...,...,...,...,...,...
180,5e9d0d95eda69973a809d1ec,631614d7ffc78f3b85670716,5e9e4502f509094188566f88,"{'core': '61c1ef45a4a2462678cbf45d', 'flight': 2, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecb075134e7cd'}",181,2022-08-28T02:22:00.000Z
181,5e9d0d95eda69973a809d1ec,630f63bf18702d4844fb5391,5e9e4502f509092b78566f87,"{'core': '5f57c54a0622a633027900a1', 'flight': 7, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3032383ecb6bb234e7ca'}",182,2022-08-31T05:40:00.000Z
184,5e9d0d95eda69973a809d1ec,63161699ffc78f3b85670719,5e9e4501f509094ba4566f84,"{'core': '60b800111f83cc1e59f16438', 'flight': 6, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecbb9e534e7cc'}",185,2022-09-17T01:05:00.000Z
185,5e9d0d95eda69973a809d1ec,631616a7ffc78f3b8567071a,5e9e4501f509094ba4566f84,"{'core': '627843d657b51b752c5c5a53', 'flight': 4, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecbb9e534e7cc'}",186,2022-09-24T23:30:00.000Z


In [74]:
# We also want to convert the date_utc to a datetime datatype and then extracting the date leaving the time
data['date'] = pd.to_datetime(data['date_utc']).dt.date

In [75]:
data

,rocket,payloads,launchpad,cores,flight_number,date_utc,date
0,5e9d0d95eda69955f709d1eb,5eb0e4b5b6c3bb0006eeb1e1,5e9e4502f5090995de566f86,"{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",1,2006-03-24T22:30:00.000Z,2006-03-24
1,5e9d0d95eda69955f709d1eb,5eb0e4b6b6c3bb0006eeb1e2,5e9e4502f5090995de566f86,"{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",2,2007-03-21T01:10:00.000Z,2007-03-21
3,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e5,5e9e4502f5090995de566f86,"{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",4,2008-09-28T23:15:00.000Z,2008-09-28
4,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e6,5e9e4502f5090995de566f86,"{'core': '5e9e289ef359184f103b2627', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",5,2009-07-13T03:35:00.000Z,2009-07-13
5,5e9d0d95eda69973a809d1ec,5eb0e4b7b6c3bb0006eeb1e7,5e9e4501f509094ba4566f84,"{'core': '5e9e289ef359185f2b3b2628', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",6,2010-06-04T18:45:00.000Z,2010-06-04
...,...,...,...,...,...,...,...
180,5e9d0d95eda69973a809d1ec,631614d7ffc78f3b85670716,5e9e4502f509094188566f88,"{'core': '61c1ef45a4a2462678cbf45d', 'flight': 2, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecb075134e7cd'}",181,2022-08-28T02:22:00.000Z,2022-08-28
181,5e9d0d95eda69973a809d1ec,630f63bf18702d4844fb5391,5e9e4502f509092b78566f87,"{'core': '5f57c54a0622a633027900a1', 'flight': 7, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3032383ecb6bb234e7ca'}",182,2022-08-31T05:40:00.000Z,2022-08-31
184,5e9d0d95eda69973a809d1ec,63161699ffc78f3b85670719,5e9e4501f509094ba4566f84,"{'core': '60b800111f83cc1e59f16438', 'flight': 6, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecbb9e534e7cc'}",185,2022-09-17T01:05:00.000Z,2022-09-17
185,5e9d0d95eda69973a809d1ec,631616a7ffc78f3b8567071a,5e9e4501f509094ba4566f84,"{'core': '627843d657b51b752c5c5a53', 'flight': 4, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecbb9e534e7cc'}",186,2022-09-24T23:30:00.000Z,2022-09-24


In [76]:
# Using the date we will restrict the dates of the launches
data = data[data['date'] <= datetime.date(2020, 11, 13)]

In [77]:
data

,rocket,payloads,launchpad,cores,flight_number,date_utc,date
0,5e9d0d95eda69955f709d1eb,5eb0e4b5b6c3bb0006eeb1e1,5e9e4502f5090995de566f86,"{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",1,2006-03-24T22:30:00.000Z,2006-03-24
1,5e9d0d95eda69955f709d1eb,5eb0e4b6b6c3bb0006eeb1e2,5e9e4502f5090995de566f86,"{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",2,2007-03-21T01:10:00.000Z,2007-03-21
3,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e5,5e9e4502f5090995de566f86,"{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",4,2008-09-28T23:15:00.000Z,2008-09-28
4,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e6,5e9e4502f5090995de566f86,"{'core': '5e9e289ef359184f103b2627', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",5,2009-07-13T03:35:00.000Z,2009-07-13
5,5e9d0d95eda69973a809d1ec,5eb0e4b7b6c3bb0006eeb1e7,5e9e4501f509094ba4566f84,"{'core': '5e9e289ef359185f2b3b2628', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",6,2010-06-04T18:45:00.000Z,2010-06-04
...,...,...,...,...,...,...,...
101,5e9d0d95eda69973a809d1ec,5ef6a4600059c33cee4a829e,5e9e4502f509094188566f88,"{'core': '5ef670f10059c33cee4a826c', 'flight': 2, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3032383ecb6bb234e7ca'}",102,2020-09-03T12:46:00.000Z,2020-09-03
102,5e9d0d95eda69973a809d1ec,5ef6a48e0059c33cee4a829f,5e9e4502f509094188566f88,"{'core': '5e9e28a7f3591817f23b2663', 'flight': 3, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3032383ecb6bb234e7ca'}",103,2020-10-06T11:29:00.000Z,2020-10-06
103,5e9d0d95eda69973a809d1ec,5ef6a4d50059c33cee4a82a1,5e9e4502f509094188566f88,"{'core': '5e9e28a6f35918c0803b265c', 'flight': 6, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3032383ecb6bb234e7ca'}",104,2020-10-18T12:25:00.000Z,2020-10-18
104,5e9d0d95eda69973a809d1ec,5ef6a4ea0059c33cee4a82a2,5e9e4501f509094ba4566f84,"{'core': '5ef670f10059c33cee4a826c', 'flight': 3, 'gridfins': True, 'legs': True, 'reused': True, 'landing_attempt': True, 'landing_success': True, 'landing_type': 'ASDS', 'landpad': '5e9e3033383ecbb9e534e7cc'}",105,2020-10-24T15:31:00.000Z,2020-10-24


from the ' rocket ' we will learn about the booster name

from the ' payload ' we will learn about the mass of the payload and the orbit that it is going to

from the ' launchpad ' we will know the name of the launch site being used, the longitude and the latitude

from the ' cores ' we will learn about the outcomes of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.

 the data from these requests will be stored in lists and will be used to create a new dataframe

In [80]:
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

# these functions wiill appply teh outputs globallu to the above variables.

In [81]:
# for refrence lets check whetherablve lists are empty or not

BoosterVersion

[]

Now lets apply above defined functions one by one

In [82]:
getBoosterVersion(data)

In [83]:
BoosterVersion[0:5]

[['Falcon 1'], ['Falcon 1'], ['Falcon 1'], ['Falcon 1'], ['Falcon 9']]

In [90]:
getLaunchSite(data)

In [97]:
LaunchSite[0:5]
Latitude[0:5]
Longitude[0:5]

[167.7431292, 167.7431292, 167.7431292, 167.7431292, -80.577366]

#### Finally now lets construct our dataset using the data we have obtained from the functions

In [110]:
launch_data = { 'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}


In [111]:
launch_data

{'FlightNumber': [1,
  2,
  4,
  5,
  6,
  8,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  57,
  58,
  59,
  60,
  61,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  76,
  78,
  79,
  80,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  100,
  101,
  102,
  103,
  104,
  105,
  106],
 'Date': [datetime.date(2006, 3, 24),
  datetime.date(2007, 3, 21),
  datetime.date(2008, 9, 28),
  datetime.date(2009, 7, 13),
  datetime.date(2010, 6, 4),
  datetime.date(2012, 5, 22),
  datetime.date(2013, 3, 1),
  datetime.date(2013, 9, 29),
  datetime.date(2013, 12, 3),
  datetime.date(2014, 1, 6),
  datetime.date(2014, 4, 18),
  datetime.date(2014, 7, 14),
  datetime.date(2014, 8, 5),
  datetime.date(2014, 9,

In [112]:
for key, value in launch_data.items():
    print(f"Length of {key}: {len(value)}")

# the code snippet is used to check the length of values of ' launch_data '

Length of FlightNumber: 94
Length of Date: 94
Length of BoosterVersion: 94
Length of PayloadMass: 0
Length of Orbit: 0
Length of LaunchSite: 96
Length of Outcome: 0
Length of Flights: 0
Length of GridFins: 0
Length of Reused: 0
Length of Legs: 0
Length of LandingPad: 0
Length of Block: 0
Length of ReusedCount: 0
Length of Serial: 0
Length of Longitude: 94
Length of Latitude: 94


In [108]:
max_length = max(len(arr) for arr in launch_data.values())
launch_data = {k: v + [None] * (max_length - len(v)) for k, v in launch_data.items()}

# the above code snippet is used because the launch_data have different array length with wich we cannot buld pandas datafram so we are replacing them with ' None ' ' value

launch_data_df = pd.DataFrame.from_dict(launch_data)

In [105]:
launch_data_df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1.0,2006-03-24,[Falcon 1],None,None,Kwajalein Atoll,None,None,None,None,None,None,None,None,None,167.743129,9.047721
1,2.0,2007-03-21,[Falcon 1],None,None,Kwajalein Atoll,None,None,None,None,None,None,None,None,None,167.743129,9.047721
2,4.0,2008-09-28,[Falcon 1],None,None,Kwajalein Atoll,None,None,None,None,None,None,None,None,None,167.743129,9.047721
3,5.0,2009-07-13,[Falcon 1],None,None,Kwajalein Atoll,None,None,None,None,None,None,None,None,None,167.743129,9.047721
4,6.0,2010-06-04,[Falcon 9],None,None,Kwajalein Atoll,None,None,None,None,None,None,None,None,None,-80.577366,28.561857


#### removing tje Falcon 1 launches keeping only the Falcone 9 launches